In [ ]:
import pandas as pd
import numpy as np

# Current way to calculate polynomials for graph overlays - replaces polyfit
from numpy.polynomial import Polynomial 

import streamlit as st
import matplotlib.pyplot as plt
from pathlib import Path

#can drop this if you want popups you can resize
%matplotlib inline 

In [ ]:
file_path = r'C:\Users\justi\Documents\Projects\Data\Baseball_databank'

teams_path = file_path + '\Teams.csv'
salary_path = file_path + '\Salaries.csv'
batting_path = file_path + '\Batting.csv'

In [ ]:
teams_df = pd.read_csv(teams_path)
teams_df = teams_df[teams_df['yearID'] >= 1985]
teams_df.info()

In [ ]:
salary_df = pd.read_csv(salary_path)
salary_df = salary_df[salary_df['yearID'] >= 1985]
salary_df.info()

In [ ]:
# How have salaries evolved by division over the years?
# Sum of salaries by division by year, then dig in by team

# .reset_index() converts back to a pandas dataframe, otherwise it would have been a series

team_salary_by_year_df = salary_df.groupby(['yearID','teamID','lgID'])['salary'].sum().reset_index()
team_salary_by_year_df.head()

In [ ]:
# cool reference material for graphs here:
# https://medium.com/dataexplorations/reproducing-the-baseball-salary-visualization-from-fivethirtyeight-in-matplotlib-1af449beb409
# Lets try and recreate!

team_salary_by_year_df['salary_zscore'] = team_salary_by_year_df.groupby(['yearID'])['salary'].transform(
    lambda x: (x - x.mean()) / x.std()
)

team_salary_by_year_df.head()

In [ ]:
teams_df.head()

In [ ]:
#now, lets get win rates. First step is grabbing season wins and losses from the teams table and cleaning it up
copy_teams_df = teams_df[['yearID','lgID','teamID','G','W','L']]
copy_teams_df = copy_teams_df[(teams_df['yearID'] >= 1985)].reset_index().drop('index',axis=1)
copy_teams_df.head()

In [ ]:
copy_teams_df.info()

In [ ]:
# Now that we have wins and total # games (W and G)
copy_teams_df['win_rate'] = copy_teams_df['W']/copy_teams_df['G']
copy_teams_df.head()

In [ ]:
merged_df = team_salary_by_year_df.merge(copy_teams_df, on = ['yearID','lgID', 'teamID']).reset_index().drop('index',axis = 1)
merged_df.head()

In [ ]:
# Lets start with one scatterplot for the Phillies (go Matt). THink it's PHI but lets check
a = merged_df['teamID'].unique()
print(sorted(a))

In [ ]:
phillies = merged_df[(merged_df['teamID'] == 'PHI')].reset_index().drop('index', axis = 1)
phillies.head()

In [ ]:
# Now, lets graph this!
plt.scatter(phillies['salary_zscore'],phillies['win_rate'], alpha = 0.5)
plt.title("PHI")
plt.xlabel("Standardized Salaries")
plt.ylabel("Win Rate")

#plotting a grid axes (win rate at y=0.5, Salary zscore at x=0)
# this also looks like it forces the graph space to be square and centered somewhat - cool
plt.hlines(0.5,-2,2)
plt.vlines(0,0.4,0.6)

# I want to add a line of best fit to the graph for the win rate. Going to use the polyfit function in NumPy, although this is outdated. Come back and update
z = np.polyfit(phillies['salary_zscore'],phillies['win_rate'], 1)
p = np.poly1d(z)

plt.plot(phillies['salary_zscore'], p(phillies['salary_zscore']),alpha=0.8,c ='#1f77b4',linewidth =3)

plt.show()

In [ ]:
# Lets do this again, but tweaking a bit to remove the border and add a gridline.
# Also, switching from deprecated polyfit method to NumPy's polynomial.Polynomial method
# Finally, remove the border on the graph using teh ax.spines method

# not sure how declaring fig, ax and using subplots allowes the use of spines 
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(x=phillies['salary_zscore'], y=phillies['win_rate'], alpha=0.5)
plt.title('PHI')
plt.ylabel('Win Rate')
plt.xlabel('Standardized Salaries')

plt.hlines(0.5, -2, 2)
plt.vlines(0,0.3,0.7)

# To create my polynomial overlay and set it's visual appearance
degree = 2
polynomial = np.polynomial.Polynomial.fit(phillies['salary_zscore'], phillies['win_rate'], degree)
ax.grid(color='grey', linestyle ='-', linewidth=0.25, alpha=0.5)

# Remove all spines at once from the graph
for spine in ax.spines.values():
    spine.set_visible(False)

plt.plot(phillies['salary_zscore'], polynomial(phillies['salary_zscore']),alpha=0.8,c ='#1f77b4',linewidth =3)

plt.show()

In [ ]:
# Print the polynomial and equation
print(f"Polynomial: {polynomial}")
print(f"Coefficients: {polynomial.coef}")

# Just for fun, what's the R-Squared for this estimate?
y = phillies['salary_zscore']
y_pred = polynomial(y)
ss_res = np.sum((y - y_pred) ** 2)
ss_tot = np.sum((y - np.mean(y)) ** 2)
r_squared = 1 - (ss_res / ss_tot)
print(f"R-Squared: {r_squared}")

# Odd NumPy doesn't have an R-squared method built in. Sklearn.metrics r2_score

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=5,figsize=(15,5),sharex=True, sharey=True)

In [ ]:
# Now, lets make a function that lets us do this for any team we want

def plot_team(team_df, ax, team_color):
    '''
        This creates a subplot for 1 team.
        team_df is the dataframe with info for that team
        ax lets us specify the subplot location
        team_color is for, well, the team color
    '''
    # Create your subplots so that the function knows where to put things
    # fig, ax = plt.subplots(figsize=(5, 5))
    # NOTE: Taking this out as when you call it later when plotting all teams in league, it overrides the subplot there. Only un-comment for testing
    
    #generate line of best fit & plot
    z = np.polynomial.Polynomial.fit(team_df['salary_zscore'],team_df['win_rate'], 1)
    ax.plot(team_df['salary_zscore'],z(team_df['salary_zscore']), alpha = 0.8, c=team_color, linewidth = 3)

    # Plot zscore and win rate
    ax.scatter(x=team_df['salary_zscore'], y=team_df['win_rate'], alpha=0.5, c=team_color)
    ax.set_title(f"{team_df['teamID'].values[0]}")
    
    # Set axes lines and reduce tickmarks
    ax.hlines(0.5,-2.5,2.5,'k')
    ax.vlines(0,0.3,0.7,'k')
    ax.set_xticks([-2,0,2])

    # Removing spines with code from earlier
    for spine in ax.spines.values():
        spine.set_visible(False)
    
    # Set grid visuals
    ax.grid(color='grey', linestyle='-', linewidth=0.25, alpha=0.5)

## **NOTE:** The following 2 cells below won't run unless you un-comment line 11 in the cell above

In [ ]:
#plot_team(merged_df[(merged_df['teamID']=='ATL')],ax,)

In [ ]:
al_east_team_dict={
    'BAL':'#FF6F1C', 
    'BOS':'#FFB627',
    'NYA':'#223843', 
    'TOR':'#1f77b4', 
    'FLO':'#FFCAD4'
}
for team, color in al_east_team_dict.items():
    plot_team(merged_df[(merged_df['teamID']==team)],ax,color)

In [ ]:
# create a plot that's 1 row and 5 columns wide
# sharex=True, sharey=True: these plots will share the same x and y axis
fig, ax = plt.subplots(nrows=1, ncols=5,figsize=(15,5),sharex=True, sharey=True)
# set an overall title for the plot
fig.suptitle('American League East', ha='center',color = 'grey',fontsize=20,va='top',) 
# help the subplots fit better
fig.subplots_adjust(hspace = .5, wspace=.001)

pos=0 # determines which axes object we will pass into the function to be filled by the team's plot
for team,color in al_east_team_dict.items():
    plot_team(merged_df[(merged_df['teamID']==team)], ax[pos],color)
    pos+=1
    
# Force label to appear on first plot rather than the last plot
ax[0].set(xlabel='Salary Z-scores', ylabel='Winning Percentage')

In [ ]:
# Now, lets do this for all leagues and teams!!
al_central_team_dict={
    'CLE':'#FF6F1C', 
    'MIN':'#FFB627',
    'DET':'#223843', 
    'CHA':'#1f77b4', 
    'KCA':'#FFCAD4'
}
al_west_team_dict={
    'HOU':'#FF6F1C', 
    'OAK':'#FFB627',
    'SEA':'#223843', 
    'LAN':'#1f77b4', 
    'TEX':'#FFCAD4'
}

nl_central_team_dict={
    'ML4':'#FF6F1C', 
    'CHN':'#FFB627',
    'SLN':'#223843', 
    'PIT':'#1f77b4', 
    'CIN':'#FFCAD4'
}
nl_west_team_dict={
    'LAN':'#FF6F1C', 
    'COL':'#FFB627',
    'ARI':'#223843', 
    'SFN':'#1f77b4', 
    'SDN':'#FFCAD4'
}
nl_east_team_dict={
    'ATL':'#FF6F1C', 
    'WAS':'#FFB627',
    'PHI':'#223843', 
    'NYN':'#1f77b4', 
    'MIA':'#FFCAD4'
}

league_list = [al_east_team_dict, al_central_team_dict,al_west_team_dict, nl_east_team_dict, nl_central_team_dict, nl_west_team_dict]

In [ ]:
# To fit everyone, we need a subplot that's 6 rows and 5 columns
fig, ax = plt.subplots(nrows=6, ncols=5,figsize=(15,15),sharex=True, sharey=True)

# set a title for the whole plot
fig.suptitle('MLB: Spend more, Win more?', ha='center',color = 'grey',fontsize=20,va='top',) 

# help the subplots fit better
fig.subplots_adjust(hspace = .5, wspace=.001)

#ax[count[pos]:  the axes object contains a numpy array for each row with 5 sub plots (count indicates the row and pos indicates the column)
count = 0 # sets the row to be filled 
for league in league_list:
    pos=0 # sets the column to be filled by a given team in the division
    for team,color in league.items():
        plot_team(merged_df[(merged_df['teamID']==team)], ax[count][pos],color)
        pos=pos+1
    count += 1

## Predictions

Cool! Now, lets actually try and *predict* win rates here, since that's ultimately what we want to do with this project. Since I don't know much about baseball, looking to this resource for a few ideas.

https://www.samford.edu/sports-analytics/fans/2022/MLB-Winning-Percentage-Breakdown-Which-Statistics-Help-Teams-Win-More-Games

I'm going to start off with the 'Pythagorean Theorem of Baseball', or run differential. TLDR: outscore your opponents, win the game. Seems simnple enough. Right? Run differential is simply the number of runs scored less runs allowed. This is in the Teams.csv as RA and R.

In [ ]:
teams_df.info()

In [ ]:
teams_df['run_differential'] = teams_df['R'] - teams_df['RA']
teams_df.info()

In [ ]:
teams_df_trimmed = teams_df[teams_df['yearID']==2015].reset_index().drop('index',axis = 1)
teams_df_trimmed.head()

In [ ]:
plt.scatter(teams_df_trimmed['W'],teams_df_trimmed['run_differential'], alpha = 0.75)

plt.title('Wins vs. Run Diff')
plt.xlabel('Wins')
plt.ylabel('Run Diff')
plt.hlines(0,65,100)

# Lets fit a line and output the coefs 

polynomial = np.polynomial.Polynomial.fit(teams_df_trimmed['W'], teams_df_trimmed['run_differential'], 1)
plt.plot(teams_df_trimmed['W'], polynomial(teams_df_trimmed['W']), alpha = 0.8, c= '#FF694D')

#plt.plot(phillies['salary_zscore'], polynomial(phillies['salary_zscore']),alpha=0.8,c ='#1f77b4',linewidth =3)

plt.show()

If it wasn't obvious before, scoring more than your opponent is a pretty good indicator on if you'll win. What about pitching? Does better pitching (like fewer runs allowed) lead to higher win rates than good hitting?